In [ ]:
%matplotlib inline

# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv`. This is the dataset to use for this assignment. Note: The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Ann Arbor, Michigan, United States**, and the stations the data comes from are shown on the map below.

In [ ]:
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd

def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))

    station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

    return mplleaflet.display()

leaflet_plot_stations(400,'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89')

In [ ]:
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd
import numpy as np

def leaflet_temperature():

    df = pd.read_csv('data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv')
    df = df.sort_values(['ID', 'Date'])
    df['Data_Value'] = df['Data_Value']*0.1
    
    df['Year'] = df['Date'].apply(lambda x: x[:4])
    df['Month-Day'] = df['Date'].apply(lambda x: x[-5:])
        
    df = df.loc[df['Month-Day'] != '02-29']
    
    index = df['Month-Day'].unique()
    
    df_hist = df[(df['Year'] >= '2005') & (df['Year'] < '2015')]
    df_15 = df[df['Year'] == '2015']
    
    max_hist = df_hist.groupby('Month-Day').agg({'Data_Value': np.max}).reset_index()
    min_hist = df_hist.groupby('Month-Day').agg({'Data_Value': np.min}).reset_index()
    
    max_15 = df_15.groupby('Month-Day').agg({'Data_Value': np.max})
    min_15 = df_15.groupby('Month-Day').agg({'Data_Value': np.min})
    
    rec_max = pd.merge(max_hist.reset_index(), max_15.reset_index(), left_index = True, on = 'Month-Day')
    rec_min = pd.merge(min_hist.reset_index(), min_15.reset_index(), left_index = True, on = 'Month-Day')
    
    max_break = rec_max[rec_max['Data_Value_y'] > rec_max['Data_Value_x']]
    min_break = rec_min[rec_min['Data_Value_y'] < rec_min['Data_Value_x']] 
    
    plt.figure(figsize=(10,5))  
    plt.plot(max_hist.index, max_hist['Data_Value'], color = 'r')
    plt.plot(min_hist.index, min_hist['Data_Value'], color = 'b')
    
    plt.scatter(max_break.index, max_break['Data_Value_y'], color='red', linewidths=3)
    plt.scatter(min_break.index, min_break['Data_Value_y'], color='g', linewidths=3)
    
    ax = plt.gca()
    ax.set_xlabel('Month-Day')
    ax.set_ylabel('Temperature [°C]')
    ax.set_title('Record Temperatures by day of the year over 2005-2014 in Ann Arbor, Michigan, United States with 2015 records breaks')
    ax.legend(['T_max', 'T_min', 'Break_max', 'Break_min'])
    
    date1 = rec_min.index[0]
    date2 = rec_min.index[-1]
    
    ax.axis([date1,date2,-50,50])
    
    ax.fill_between(min_hist.index, max_hist['Data_Value'], min_hist['Data_Value'], facecolor='blue', alpha=0.2)
    plt.xticks(range(0,365, 12), index[0:-1:12])
    
    for item in ax.xaxis.get_ticklabels():
        item.set_rotation(45)
    
    plt.subplots_adjust(bottom=0.25)
    
    plt.savefig('Assignment 2 - Figure.png')
                         
    return plt.show()